In [ ]:
"""
Author: Matt Martin
Date: 1/27/24
Desc: Various techniques to generate test data in spark
"""

## create the spark connection/instance
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("sequence") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.instances", 10) \
    .getOrCreate()

In [ ]:
#generate row sequence
df = spark.range(1,1_000_001).toDF('row_id')

## the sql version is very slow; the spark.range version is very fast
#df = spark.sql("SELECT EXPLODE(SEQUENCE(1, 1000000)) AS row_id")
#generate a list of dates
start_dt = "2001-01-01"
end_dt = "2024-05-10"
dates = spark.sql(f"SELECT explode(sequence(to_date('{start_dt}'), to_date('{end_dt}'), interval 1 day)) as date")

In [ ]:
df.createOrReplaceTempView('numbers')
dates.createOrReplaceTempView('dates')

In [ ]:
sql = """
select n.row_id, d.date, uuid() as test_id
from numbers as n, dates as d
limit 10000
"""
spark.sql(sql).show(10)

In [ ]:
spark.stop